#### 1.

Przypomnij sobie z wykładu, w jaki sposób możemy łączyć klasyfikatory ze sobą (kilka architektur) oraz co jest niezbędne (jakie warunki muszą być spełnione) do tego, żeby takie połączenia działały skuteczniej od ich elementów składowych.

Nawiązując do informacji z wykładu przeczytaj dokumentację pakietu scikit-learn na temat metod zespołowych w klasyfikacji (pomiń regresję; skup się tylko na BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier i StackingClassifier).

Pobierz zbiór danych o nazwie odpowiadającej Twojemu numerowi albumu i przeprowadź jego wstępną eksplorację: 
1. proporcja klas, 
1. liczba i rodzaje atrybutów, 
1. ich zakresy i rozkłady wartości. 

Pokaż rozkłady wartości wszystkich atrybutów obok siebie na jednym szerokim wykresie pudełkowym lub skrzypcowym; na osi poziomej umieść nazwy atrybutów. Opisując wnioski (wystarczy kilka zdań) możesz pogrupować (o ile to możliwe) atrybuty pisząc np. "73 atrybuty są takie a takie, 22 atrybuty charakteryzują się tym a tym, wyjątkowy jest atrybut taki a taki", itp.